In [33]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import pathlib

def evaluate(pathGoldenLabelsFile):
       
    goldenLabels = [label.strip() for label in open(pathGoldenLabelsFile).readlines()]     
    goldenClusters = {}
    ind = -1        
    
    for label in goldenLabels:
        if label in goldenClusters:
            ind = goldenClusters[label]
        else:
            ind += 1
            goldenClusters[label] = ind                                            
            
    with open("results/theta.accuracy", "w") as accuracy_file: 
        accuracy_file.write("Golden-labels in: " + pathGoldenLabelsFile + "\n\n")
        accuracies = []

        for filepath in pathlib.Path("results").glob('**/*'):            
            if not filepath.name.endswith("theta"):
                continue  
            docsTopicProbs = open(filepath).readlines()
            if len(docsTopicProbs) != len(goldenLabels):
                print("Error: the number of documents is different to the number of labels!")
                raise Exception() 
                    
            y = np.array([goldenClusters[goldenLabels[i]] for i in range(len(docsTopicProbs))])
            X = np.array([np.fromstring(docTopicProb, sep=' ') for docTopicProb in docsTopicProbs])
 
            accuracy = cross_val_score(LinearSVC(), X, y, cv=5)
                    
            accuracy_file.write("Results for: " + str(filepath) + "\n")                                 
            accuracy_file.write("\tAccuracy: " + str(accuracy) + "\n")
            accuracies.append(accuracy)
        
        if len(accuracies) == 0:
            print("Error: There is no file ending with theta")
            raise Exception()        
        
        accuracies = np.array(accuracies)
        accuracy_file.write("\n---\nMean accuracy: " + str(np.mean(accuracies))
                + ", standard deviation: " + str(np.std(accuracies)))

        print("\n---\nMean accuracy: " + str(np.mean(accuracies))
                + ", standard deviation: " + str(np.std(accuracies)))
